# Multiplicative Weights and Boosting

This notebook is going to take a relatively different approach to introducing boosting. Instead of introducing boosting within the scope of some machine learning application such as decision trees (which was assumedly covered in a prior week), we will first look more broadly at the general algorithmic framework of multiplicative weights, which has wide applications in fields as diverse as game theory and finance. Then, we will go back to boosting through decision trees and random forests. 

The idea of aggregating information from many sources has great utility in machine learning and is the basis behind ensemble methods. Boosting is one example of an ensemble method which has had great success in real-life applications. Indeed, the fundamental ideas behind these concepts are universal, and you have seen them in previous classes such as EECS16B through Lagrange Interpolation and Orthogonal Matching Pursuit (OMP). First, let's explore the general experts framework and introduce the multiplicative weights algorithm.


In [ ]:
# Imports here 
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier

## An Experts Framework
In many scenarios, we may have different models or "experts" that are telling us how to go about performing some task. The experts may give conflicting pieces of advice. How should we go about using their advice?

In the simplest, one-day scenario, without any prior on the experts, we would equally consider the advice from each of the experts... In other words, for a discrete classification problem we simply choose the majority rule.

As a warmup, let's consider a classic scenario where we are trying to classify a handwritten digit, 0 through 9. Suppose we have $n$ experts who give us their opinion on a single data point. Write a function that will take the majority vote for this case.

In [ ]:
# Function that returns the majority rule based off the 
# experts advice. Takes in a list containing each 
# expert's decision and an int representing the number of experts n,
# and returns the digit corresponding to the majority.
def discreteClassificationMajority(experts, n):
  # maxFreq = 0
  # maxIndex = experts[0] 
  # for i in experts: 
  #     freq = experts.count(i) 
  #     if freq > maxFreq: 
  #         maxFreq = freq 
  #         maxIndex = i
  # return i

  return max(set(experts), key = experts.count) 

n = 5
experts1 = [0, 2, 2, 5, 2]
experts2 = [0, 0, 2, 5, 7]
print(discreteClassificationMajority(experts1, n)) # should return 2.
print(discreteClassificationMajority(experts2, n)) # should return 0.


#PERHAPS CODE UP CONTINUOUS (JUST AN AVERAGE) HERE!

2
0


Now, suppose in the discrete case, the experts had more information than just their first choice decision? Specifically, what if the experts rank the preferences? Two possible methods we could do to take advantage of this extra information are: 

1. Create a separate weight, or a "score," to each category based on the position it is in the expert's list and take the linear combination of the scores with the choices. 


2. Do instant runoff voting (keep eliminating the last place and redistributing their votes to the next preference until some candidate has a 50% majority). 

We will omit the code for time's sake due to added complexity from the redistribution of votes)




Question 1:

What is an advantage of method 1? What is a scenario in which method 1. gives us a result that the original majority vote would not?

Answer 1:

Method 1 gives us more details on the entire classification rather than just the best choice. So, classes other than just the majority class also have impact.

Thus, an example of an instance in which this could lead to more accurate results is: suppose that the majority vote is a tie, or near tie, between multiple classes. But then, suppose one of these majority classes has significantly more votes from experts as the second choice or other near-the-top choices. Then, that class will be rewarded in this method and is more likely to be the best class indeed, while the simple majority vote does not have the depth to foresee this scenario. Say, candidate A narrowly beats candidate B for a majority, but candidate B has more voters for second place and third place than candidate A, thus surpassing A.


Question 2:

What is an advantage of method 2? What is a scenario in which method 2. gives us a result that the original majority vote would not? 

Answer 2:


Advantage 2 gives us more details on what the experts who didn't pick the majority would have chosen for their top choices, given the opportunity to do so. Thus, we can have a more informed choice at each later stage on the remaining candidates all the way until there is only two classes and every expert is voting on which one or the other. Thus, an example of an instance in which this could lead to more accurate results is when: suppose Candidate A narrowly beats Candidate B for the majority initially, but once the votes are redistributed from bottom up, candidate B gets more of those experts' votes and surpasses candidate A.


## Multiplicative Weights Algorithm

Back to experts scenario. What about if we have multiple days. Based on the outcome from the previous days until now. Some experts may be more skilled (i.e making accurate decisions more frequently) than others. Thus, we want to weigh their advice more comparatively. This is where the multiplicative weights update algorithm comes in.




In [ ]:
# simple example of what the experts tell you to do 
# Example with day trading or betting on horse races.
# code up weighted avg and weight update method (may use higher order functions)

What if we had one "perfect expert" who always predicts correctly. How could we use that to improve our accuracy over time?

Answer:

You could construct a halving algorithm - each day, if we make a mistake, throw out all that advised you wrongly. Make maximum of log_2(x) mistakes since by majority rule, at most half are incorrect!


In the real world, no one is perfect. So, we usually can't just throw out anyone who makes a single mistake, since pretty soon we'd end up with no one left to give us advice! So instead, we discount the incorrect experts, but count them again...

Here, we'd like to update the the experts' weights in a way such that correctness is rewarded and incorrectness is punished.

From here, we will probabilistically make a decision. Experts with higher weight have more probability mass, and vice versa.

# Ensemble Methods

Now equipped with the multiplicative weights framework, let us explore the applications of this into machine learning, through ensemble methods. Here we will use sklearn to implement random forests and adaboost.

# Step 1: Creating the data set

Sklearn has nice data sets we can use for this. For example, let us explore the moons data set. Here we get the data and partition it into training and testing.

In [ ]:
X, y = make_circles(n_samples=10000, noise=.5, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Random Forests (Bagging)

First, let us use a bagging method that was introduced to you previously, random forests. 


# New Section

In [ ]:
classifier = RandomForestClassifier(n_estimators=100, max_features="auto",random_state=0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
accuracy_score(y_test, y_pred)

0.529

## Boosting 

Now, let's think back to decision trees. we can apply this multiplicative weight update method to our decision trees, where the "experts" are our data points, and we upweight the misclassified points then run DTL lagorithm, repeatedly, combining the trees from each round

Metacognitive whoa!

In [ ]:
classifier = AdaBoostClassifier(n_estimators=100)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
accuracy_score(y_test, y_pred)

0.5595

# **Conclusion**

Thus, ensemble methods have their place as a fundamental range of techniques in machine learning. As the adage goes, the more the merrier. By intelligently combining the expertise of multiple models, we can surpass old limits. Now, you understand from a general sense, the intuition of ensemble methods by exploring the multiplicative weight updates algorithm and framework and how it applies to machine learning.


# Optional:

Furthermore, adaboost may be generalized to gradient boosting. You may code that method up here and compare its performance to our previous methods!

In [ ]:
classifier = GradientBoostingClassifier(n_estimators=100)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
accuracy_score(y_test, y_pred)


0.5625